# DB2-Salesforce connector: Basic tool information updates

In [1]:
# Parameters
api_url = '/services/data/v43.0/sobjects'
external_id = 'Tool_name__c'
object_id = 'nanoHUB_tools__c'

# login parameters to be handled by Papermill
sql_login_params = {"username": "wang2506_ro", "password": "fnVnwcCS7iT45EsA"}
sf_login_params = {
    "grant_type": "password",
    "client_id": "3MVG95jctIhbyCppj0SNJ75IsZ1y8UPGZtSNF4j8FNVXz.De8Lu4jHm3rjRosAtsHy6qjHx3i4S_QbQzvBePG",
    "client_secret": "D1623C6D3607D4FC8004B92C761DFB6C1F70CCD129C5501E357028DFA00F5764",
    "username":"wang2506@purdue.edu",
    "password":"mathisfun992728tXYwCb5yMm6XtohbTRw8ZdBbi",    
} 
    #"username": "wang159-4j1v@force.com",
    #"password": "napoleon0eZ3PQpQqE3C3z4wWqEqKGhQ8",

In [3]:
import sys
sys.path.append('/home/users/wang2506/nanohub_salesforce_integ/salesforce')

import pandas as pd
import datetime

## Obtain tool information from DB2 

In [4]:
# Basic tool information
sql_query = "select toolname, title, published, registered from jos_tool"

# display
print(sql_query)

select toolname, title, published, registered from jos_tool


In [5]:
# connect with DB2
import sqlalchemy as sql

engine = sql.create_engine('mysql+pymysql://%s:%s@127.0.0.1/nanohub' \
                                               %(sql_login_params['username'], sql_login_params['password']))
df = pd.read_sql_query(sql_query, engine, parse_dates=['registered'])

# display
df.head(2)

,toolname,title,published,registered
0,ellipsom,title-ellipsom,0,2005-03-05 03:02:47
1,hydrolab,Hydrophobicity Lab,1,2006-02-24 03:38:23


In [6]:
# Lifetime usage count
sql_query = "select v.toolname, count(*) as total, count(distinct t.user) as user_total from nanohub_metrics.toolstart as t \
inner join nanohub.jos_tool_version as v \
on v.instance = t.tool \
where t.user not in ('instanton', 'gridstat')\
group by v.toolname"

toolstart_lifetime_df = pd.read_sql_query(sql_query, engine)

# last year usage count
sql_query = "select v.toolname, count(*) as total, count(distinct t.user) as user_total from nanohub_metrics.toolstart as t \
inner join nanohub.jos_tool_version as v \
on v.instance = t.tool \
where t.user not in ('instanton', 'gridstat') and t.datetime >= '%s'\
group by v.toolname" % (datetime.datetime.now()-datetime.timedelta(days=365)).strftime('%Y-%m-%d')

toolstart_last_year_df = pd.read_sql_query(sql_query, engine)

# last 3-month usage count
sql_query = "select v.toolname, count(*) as total, count(distinct t.user) as user_total from nanohub_metrics.toolstart as t \
inner join nanohub.jos_tool_version as v \
on v.instance = t.tool \
where t.user not in ('instanton', 'gridstat') and t.datetime >= '%s'\
group by v.toolname" % (datetime.datetime.now()-datetime.timedelta(days=90)).strftime('%Y-%m-%d')

toolstart_last_3_month_df = pd.read_sql_query(sql_query, engine)

# last month usage count
sql_query = "select v.toolname, count(*) as total, count(distinct t.user) as user_total from nanohub_metrics.toolstart as t \
inner join nanohub.jos_tool_version as v \
on v.instance = t.tool \
where t.user not in ('instanton', 'gridstat') and t.datetime >= '%s'\
group by v.toolname" % (datetime.datetime.now()-datetime.timedelta(days=30)).strftime('%Y-%m-%d')

toolstart_last_month_df = pd.read_sql_query(sql_query, engine)

# last week usage count
sql_query = "select v.toolname, count(*) as total, count(distinct t.user) as user_total from nanohub_metrics.toolstart as t \
inner join nanohub.jos_tool_version as v \
on v.instance = t.tool \
where t.user not in ('instanton', 'gridstat') and t.datetime >= '%s'\
group by v.toolname" % (datetime.datetime.now()-datetime.timedelta(days=7)).strftime('%Y-%m-%d')

toolstart_last_week_df = pd.read_sql_query(sql_query, engine)

# last 24 hours usage count
sql_query = "select v.toolname, count(*) as total, count(distinct t.user) as user_total from nanohub_metrics.toolstart as t \
inner join nanohub.jos_tool_version as v \
on v.instance = t.tool \
where t.user not in ('instanton', 'gridstat') and t.datetime >= '%s'\
group by v.toolname" % (datetime.datetime.now()-datetime.timedelta(days=1)).strftime('%Y-%m-%d')

toolstart_last_24_hours_df = pd.read_sql_query(sql_query, engine)

In [7]:
# display
toolstart_last_year_df.head(2)

,toolname,total,user_total
0,1dchainmd,209,69
1,1dfdmht,229,83


In [8]:
# insert user and run stats into df
all_df = df.copy()

all_df = pd.merge(all_df, toolstart_lifetime_df.rename(columns={\
                                                                'total':        'Total_runs_over_lifetime',
                                                                'user_total':'Total_users_over_lifetime'}\
                                                      ), how='left', left_on='toolname', right_on='toolname')
all_df = pd.merge(all_df, toolstart_last_year_df.rename(columns={\
                                                                'total':        'Total_runs_last_year',
                                                                'user_total':'Total_users_last_year'}\
                                                      ), how='left', left_on='toolname', right_on='toolname')
all_df = pd.merge(all_df, toolstart_last_3_month_df.rename(columns={\
                                                                'total':        'Total_runs_last_3_month',
                                                                'user_total':'Total_users_last_3_month'}\
                                                      ), how='left', left_on='toolname', right_on='toolname')
all_df = pd.merge(all_df, toolstart_last_month_df.rename(columns={\
                                                                'total':        'Total_runs_last_month',
                                                                'user_total':'Total_users_last_month'}\
                                                      ), how='left', left_on='toolname', right_on='toolname')
all_df = pd.merge(all_df, toolstart_last_week_df.rename(columns={\
                                                                'total':        'Total_runs_last_week',
                                                                'user_total':'Total_users_last_week'}\
                                                      ), how='left', left_on='toolname', right_on='toolname')
all_df = pd.merge(all_df, toolstart_last_24_hours_df.rename(columns={\
                                                                'total':        'Total_runs_last_24_hours',
                                                                'user_total':'Total_users_last_24_hours'}\
                                                      ), how='left', left_on='toolname', right_on='toolname')

# display
all_df.head(5).T

,0,1,2,3,4
toolname,ellipsom,hydrolab,huckel,nanomos,sete
title,title-ellipsom,Hydrophobicity Lab,Huckel-IV,NanoMOS,SETE
published,0,1,1,1,0
registered,2005-03-05 03:02:47,2006-02-24 03:38:23,2006-03-01 18:02:30,2006-03-01 18:16:18,2006-03-01 18:16:59
Total_runs_over_lifetime,NaN,3311,NaN,98847,141
Total_users_over_lifetime,NaN,400,NaN,3489,26
Total_runs_last_year,NaN,7,NaN,15141,NaN
Total_users_last_year,NaN,4,NaN,158,NaN
Total_runs_last_3_month,NaN,1,NaN,8978,NaN
Total_users_last_3_month,NaN,1,NaN,79,NaN


## Match data with Salesforce format

In [9]:
df_sf = pd.DataFrame()

# Make sure NaN and NaT values are taken care of here

df_sf['Name']              = all_df['title']
df_sf['Tool_name__c']  = all_df['toolname']
df_sf['Tool_URL__c']     = all_df['toolname'].apply(lambda x: 'https://nanohub.org/tools/'+x)
df_sf['Published__c']    = all_df['published']

# stats
df_sf['Total_runs_over_lifetime__c']  = all_df['Total_runs_over_lifetime']
df_sf['Total_runs_last_year__c']        = all_df['Total_runs_last_year']
df_sf['Total_runs_last_3_month__c'] = all_df['Total_runs_last_3_month']
df_sf['Total_runs_last_month__c']     = all_df['Total_runs_last_month']
df_sf['Total_runs_last_week__c']       = all_df['Total_runs_last_week']
df_sf['Total_runs_last_24_hours__c'] = all_df['Total_runs_last_24_hours']

df_sf['Total_users_over_lifetime__c']  = all_df['Total_users_over_lifetime']
df_sf['Total_users_last_year__c']        = all_df['Total_users_last_year']
df_sf['Total_users_last_3_month__c'] = all_df['Total_users_last_3_month']
df_sf['Total_users_last_month__c']     = all_df['Total_users_last_month']
df_sf['Total_users_last_week__c']       = all_df['Total_users_last_week']
df_sf['Total_users_last_24_hours__c'] = all_df['Total_users_last_24_hours']

# solidify time-related columns from datetime to string
df_sf['Creation_date__c']  = all_df['registered'].dt.date.fillna('').astype('str')

df_sf.fillna(0, inplace=True)

sf_original_fields = df_sf.columns

# display
df_sf.head(2).T

,0,1
Name,title-ellipsom,Hydrophobicity Lab
Tool_name__c,ellipsom,hydrolab
Tool_URL__c,https://nanohub.org/tools/ellipsom,https://nanohub.org/tools/hydrolab
Published__c,0,1
Total_runs_over_lifetime__c,0,3311
Total_runs_last_year__c,0,7
Total_runs_last_3_month__c,0,1
Total_runs_last_month__c,0,1
Total_runs_last_week__c,0,0
Total_runs_last_24_hours__c,0,0


## To Salesforce Sales Cloud CRM

In [10]:
from DB2SalesforceAPI import DB2SalesforceAPI

# create DB2 to Salesforce API object
db_s = DB2SalesforceAPI(sf_login_params)

db_s.object_id = object_id
db_s.external_id = external_id

Obtained Salesforce access token ...... True


In [11]:
# send data to Salesforce
db_s.send_data(df_sf)

[Success] Bulk job creation successful. Job ID = 7505w00000M0kO7AAJ
[Success] CSV upload successful. Job ID = 7505w00000M0kO7AAJ
[Success] Closing job successful. Job ID = 7505w00000M0kO7AAJ


In [12]:
# check status
db_s.check_bulk_status()

{'id': '7505w00000M0kO7AAJ',
 'operation': 'upsert',
 'object': 'nanoHUB_tools__c',
 'createdById': '0055w00000ArpYvAAJ',
 'createdDate': '2020-05-11T08:52:20.000+0000',
 'systemModstamp': '2020-05-11T08:52:21.000+0000',
 'state': 'InProgress',
 'externalIdFieldName': 'Tool_name__c',
 'concurrencyMode': 'Parallel',
 'contentType': 'CSV',
 'apiVersion': 47.0,
 'jobType': 'V2Ingest',
 'lineEnding': 'LF',
 'columnDelimiter': 'COMMA',
 'numberRecordsProcessed': 0,
 'numberRecordsFailed': 0,
 'retries': 0,
 'totalProcessingTime': 0,
 'apiActiveProcessingTime': 0,
 'apexProcessingTime': 0}